In [1]:
import ddddocr
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pickle
from concurrent.futures import ThreadPoolExecutor
import cv2
import numpy as np
import io
from PIL import Image
import keyboard
from IPython.display import clear_output
import re

In [2]:
def refresh_cookies():
    options = uc.ChromeOptions()
    driver = uc.Chrome(options=options)
    driver.maximize_window()
    driver.get(f'https://tixcraft.com')
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class='close-alert']"))).click()
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[id='onetrust-reject-all-handler']"))).click()
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "li[class='account-login text-center dropdown register-link d-nonex'] > a"))).click()
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[id='loginGoogle']"))).click()
    while(True):
        key = input("After login and finished all tasks, wait a few seconds and enter 'y' to save cookies\n")
        if(key == 'y'): break
    pickle.dump(driver.get_cookies(), open(f"tixcraft_cookies.pkl", "wb"))
    print('Cookies saved')
    driver.quit()
# refresh_cookies()

In [3]:
class StartupTasks:
    def __init__(self) -> None:
        global driver

    def adjust_window(self):
        driver.set_window_position(2000, 0)
        driver.maximize_window()
    
    def add_cookies(self):
        driver.delete_all_cookies()
        cookies = pickle.load(open("tixcraft_cookies.pkl", "rb"))
        for cookie in cookies:
            driver.add_cookie(cookie)

    def close_alert_button(self):
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "button[class='close-alert']"))).click()
    
    
    def execute_tasks(self):
        print('Selected activity:', driver.find_element(By.CSS_SELECTOR, "h1").text)
        with ThreadPoolExecutor() as executor:
            executor.submit(self.close_alert_button)
            executor.submit(self.add_cookies)
            executor.submit(self.adjust_window)
        driver.refresh()


In [11]:
class EventPageTasks:
    def __init__(self) -> None:
        global driver, event_priority, actions
    
    def event_selection(self, time_name_venue_status):
        datetime_e, name_e, venue_e, status_e = [cell.text for cell in time_name_venue_status]
        datetime_match, name_match, venue_match = True, True, True
        for i, (datetime_p, name_p, venue_p) in enumerate(event_priority):
            if(datetime_p):
                datetime_match = (datetime_e == datetime_p)
            if(name_p):
                name_match = (name_e == name_p)
            if(venue_p):
                venue_match = (venue_e == venue_p)
            if(datetime_match and name_match and venue_match):
                if(status_e == "Find tickets"):
                    return i
                else:
                    print(f"{datetime_e}, {name_e}, at {venue_e} not available")
        return 128
    
    def execute_tasks(self):
        found_matched_event = False
        while(not found_matched_event):

            if(keyboard.is_pressed('q')):
                break
            clear_output(wait=True)
            
            event_table = WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "tbody > tr")))
            event_rows = [WebDriverWait(row, 5).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "td"))) for row in event_table]
            event_selection_check = np.ones(len(event_rows)) * 128

            with ThreadPoolExecutor() as executor:
                event_selection_result = [priority_idx for priority_idx in executor.map(self.event_selection, event_rows)]

            if(not np.all(event_selection_result == event_selection_check)):
                event_idx = np.argmin(event_selection_result)
                datetime, name, venue, status = event_rows[event_idx]
                print(f"Found event: {datetime.text}, {name.text}, at {venue.text}")
                found_matched_event = True
                actions.move_to_element(status).perform()
                while(True):
                    try:
                        status.click()
                        break
                    except:
                        pass
            else:
                print("No available tickets, refreshing...")
                driver.refresh()

In [5]:
class SeatSelectionPageTasks:
    def __init__(self) -> None:
        global driver, wanted_section_keywords_priority, ticket_quantity, actions, manual_seating
    
    def section_selection(self, name_status):
        name, status = name_status
        for i, section_keywords in enumerate(wanted_section_keywords_priority):
            exact_match = True
            for keyword in section_keywords:
                if(keyword not in name):
                    exact_match = False
                    break
            if(exact_match and ("Sold out" not in status)):
                print(f"Section {name}, {status}")
                if("seat(s) remaining" in status):
                    if(int(re.search(r"([0-9]) seat\(s\) remaining", status).group(1)) < ticket_quantity):
                        break
                return i
        return 128
    
    def press_manual_button(self):
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "label[for='select_form_manual']"))).click()
    
    def execute_tasks(self):
        found_matched_section = False
        while(not found_matched_section):

            if(keyboard.is_pressed('q')):
                break
            clear_output(wait=True)
            
            section_li_lst = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[class='zone area-list'] > ul > li")))
            section_name_status = [s.text.strip().split(' ', 1) for s in section_li_lst]
            section_selection_check = np.ones(len(section_name_status)) * 128

            with ThreadPoolExecutor() as executor:
                section_selection_result = [priority_idx for priority_idx in executor.map(self.section_selection, section_name_status)]
                if(manual_seating):
                    executor.submit(self.press_manual_button)
            
            if(not np.all(section_selection_result == section_selection_check)):
                section_idx = np.argmin(section_selection_result)
                print(f"Found tickets: {section_name_status[section_idx][0]}, {section_name_status[section_idx][1]}")
                found_matched_section = True                    
                actions.move_to_element(section_li_lst[section_idx]).perform()
                while(True):
                    try:
                        section_li_lst[section_idx].click()
                        break
                    except Exception as e:
                        print(e)
                if(manual_seating):
                    WebDriverWait(driver, 5).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe[class='fancybox-iframe']")))
                    seats = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "empty")))
                    for i in range(ticket_quantity):
                        seats[i].click()
                        try:
                            driver.switch_to.alert.accept()
                        except:
                            pass
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[id='submitSeat']"))).click()
                    driver.switch_to.default_content()
                
            else:
                driver.refresh()

In [6]:
class TicketQtyPageTasks:
    def __init__(self) -> None:
        global driver, ticket_quantity, actions, manual_seating
    
    def enter_captcha(self):
        def guess_captcha():
            ocr = ddddocr.DdddOcr(show_ad=False)
            while(True):
                captcha = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "img[id='TicketForm_verifyCode-image']")))
                img = np.array(Image.open(io.BytesIO(captcha.screenshot_as_png)).convert('L'))
                if(img.shape == (100, 120)):
                    break
            for kernel_size in [3, 4, 2]:
                img_erode = cv2.erode(img, np.ones((kernel_size, kernel_size), np.uint8))
                guess = ocr.classification(Image.fromarray(img_erode))
                if(len(guess) == 4):
                    break
            if(len(guess) != 4):
                guess = ocr.classification(Image.fromarray(img))
            return guess
        try:
            captcha_guess = guess_captcha()
            verify_code_box = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[id='TicketForm_verifyCode']")))
            verify_code_box.clear()
            verify_code_box.send_keys(captcha_guess)
        except Exception as e:
            print(f'Enter captcha failed with {e}')
    
    def agree_tos(self):
        agree_tos_checkbox = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[id='TicketForm_agree']")))
        while(not agree_tos_checkbox.is_selected()):
            try: 
                agree_tos_checkbox.click() 
            except: 
                pass

    def select_quantity(self):
        if(manual_seating):
            return
        full_ticket = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select")))
        possible_quantity = WebDriverWait(full_ticket, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "option")))
        final_quantity = min(ticket_quantity, len(possible_quantity) - 1)
        choose_quantity = possible_quantity[final_quantity]
        try:
            while(not choose_quantity.is_selected()):
                choose_quantity.click()
        except Exception as e:
            print(f'Select quantity failed with {e}')

    def get_submit_button(self):
        return WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class='btn btn-primary btn-green']")))
    
    def execute_tasks(self):
        with ThreadPoolExecutor() as executor:
            executor.submit(self.agree_tos)
            executor.submit(self.select_quantity)
            executor.submit(self.enter_captcha)
            submit_button = executor.submit(self.get_submit_button).result()
        actions.move_to_element(submit_button).perform()
        while(True):
            try:
                submit_button.click()
                break
            except:
                pass

In [7]:
ticket_quantity = 2
manual_seating = True
event = '25_keshi_c'
url = f'https://tixcraft.com/activity/game/{event}'
event_priority = [
    [None, "【Mastercard專區】keshi：REQUIEM TOUR IN TAIPEI", None],
]
wanted_section_keywords_priority = [
    ['A13'], 
    ['A14'], 
    ['A15'], 
    ['A16'],
    ['A7'],
    ['A8'],
    ['紫2C'],
    ['A2區'],
    ['B3'],
]
print(f"Wanting to purchase \"{ticket_quantity}\" tickets")



Wanting to purchase "2" tickets


In [8]:
options = uc.ChromeOptions()
driver = uc.Chrome(options=options)
actions = ActionChains(driver)
driver.get(url)
startup_tasks = StartupTasks()
startup_tasks.execute_tasks()

Selected activity: 【Mastercard專區】keshi：REQUIEM TOUR IN TAIPEI


In [12]:
event_page_tasks = EventPageTasks()
seat_selection_page_tasks = SeatSelectionPageTasks()
ticket_qty_page_tasks = TicketQtyPageTasks()

In [16]:
while(True):
    clear_output(wait=True)
    if(keyboard.is_pressed('q')):
        break
    current_url = driver.current_url
    if(current_url == url):
        wait_seconds = 1
        event_page_tasks.execute_tasks()
    elif("area" in current_url):
        wait_seconds = 1
        seat_selection_page_tasks.execute_tasks()
    elif("ticket/ticket" in current_url):
        wait_seconds = 5
        ticket_qty_page_tasks.execute_tasks()
    elif(current_url == "https://tixcraft.com/ticket/order"):
        wait_seconds = 1
    elif("verify" in current_url):
        wait_seconds = 1
        card_input = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[class='greyInput']")))
        card_input.clear()
        card_input.send_keys("jjjjj")
        WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
    elif(current_url == "https://tixcraft.com/ticket/checkout"):
        break
    try:
        alert = WebDriverWait(driver, wait_seconds).until(EC.alert_is_present())
        print(alert.text)
        alert.accept()
    except:
        pass

The promo code that you entered is incorrect. Please try again.
